In [ ]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
from pymongo import MongoClient
from datetime import datetime

In [ ]:
driver = webdriver.Chrome()
mainUrl="https://www.cbc.ca/news/politics"
driver.get(mainUrl)

In [ ]:
soup = BeautifulSoup(driver.page_source,'html.parser')
linksout = soup.find_all('a')

In [ ]:
#linksout

In [ ]:
#Fetching all links in page.
allLinks=[]
for link in linksout:
    if(link.has_attr('href')):
        allLinks.append(link['href'])

In [ ]:
len(allLinks)

In [ ]:
#allLinks

In [ ]:
linksrequired=[]
substr = '/news/politics'
for i in allLinks:
    if(substr in i):
        linksrequired.append('https://www.cbc.ca'+i)

In [ ]:
#linksrequired

In [ ]:
linksrequired.pop()

In [ ]:
Head=[]
Desc=[]
for link in linksrequired:
    page=requests.get(link)
    if(page.status_code != 404):
        soup = BeautifulSoup(page.content,'html.parser')
        Heading = soup.find_all(class_="detailHeadline")
        cnt=0
        Head.append(Heading[cnt].get_text())
        cnt+=1
        Description = soup.find_all('p')
        counter=0
        sd=''
        for i in Description:
            line = Description[counter].get_text()
            counter+=1
            sd+=line
        Desc.append(sd)

In [ ]:
Head[0]

In [ ]:
Desc[0]

In [ ]:
updated_time = datetime.now()

In [ ]:

data = pd.DataFrame({
    "Heading" : Head,
    "Descriptions" : Desc,
    "Label" : "Real",
    "Source" : "CBC News",
    "Updated Date" : updated_time
})

In [ ]:
#data

In [ ]:
dataNoDup =data.drop_duplicates()

In [ ]:
#dataNoDup

In [ ]:
client =  MongoClient("paste your connection string here")

In [ ]:
db = client['Project']   #Replace 'Project' with your Database name
collection = db['test1'] #Replace 'test1' with your collection name  

In [ ]:
dataNoDup.reset_index(inplace=True)
data_dict = dataNoDup.to_dict("records")

In [ ]:
collection.insert_many(data_dict)